In [ ]:
# not in opendata.dc.gov but in gis

In [4]:
import csv, requests, sys, time

url = "https://maps2.dcgis.dc.gov/dcgis/rest/services/DCGIS_DATA/Violations_Parking_2023/MapServer/8/query"

params = {
    "where": "1=1",
    "outFields": "*",
    "returnGeometry": "false",
    "f": "json",
}

batch = 2000 
output = "../Data/september_2023_parking_violations.csv"

In [5]:
def supports_offset():
    p = dict(params)
    p.update({"resultOffset": 0, "resultRecordCount": 1})
    r = requests.get(url, params=p, timeout=60).json()
    return "features" in r

def fetch_batch(params):
    r = requests.get(url, params=params, timeout=120)
    r.raise_for_status()
    data = r.json()
    if "error" in data:
        raise RuntimeError(data["error"])
    return data.get("features", [])

def write_csv(rows, fieldnames):
    with open(output, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=fieldnames)
        w.writeheader()
        for row in rows:
            w.writerow(row)

all_rows = []
fieldnames = None

try:
    if supports_offset():
        offset = 0
        while True:
            params = dict(params,
                          resultOffset=offset,
                          resultRecordCount=batch,
                          orderByFields="OBJECTID")
            feats = fetch_batch(params)
            if not feats:
                break
            for ft in feats:
                attrs = ft.get("attributes", {})
                if fieldnames is None:
                    fieldnames = list(attrs.keys())
                all_rows.append(attrs)
            offset += len(feats)
            time.sleep(0.2)
    else:
        last = -1
        while True:
            params = dict(params)
            params["where"] = f"OBJECTID > {last}"
            params["orderByFields"] = "OBJECTID"
            params["resultRecordCount"] = batch
            feats = fetch_batch(params)
            if not feats:
                break
            for ft in feats:
                attrs = ft.get("attributes", {})
                if fieldnames is None:
                    fieldnames = list(attrs.keys())
                all_rows.append(attrs)
                last = max(last, attrs.get("OBJECTID", last))
            time.sleep(0.2)

    if not all_rows:
        print("No rows returned. Check your WHERE clause or layer access.")
        sys.exit(0)

    fieldnames = fieldnames or sorted({k for r in all_rows for k in r.keys()})
    write_csv(all_rows, fieldnames)
    print(f"Done. Wrote {len(all_rows)} rows to {output}")

except Exception as e:
    print("Error:", e)
    sys.exit(1)

Done. Wrote 110460 rows to ../Data/september_2023_parking_violations.csv
